<a href="https://colab.research.google.com/github/CadenChou/insider-trading-analysis/blob/main/Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install yfinance python-dateutil


In [2]:
import requests
import pandas as pd
import urllib.parse
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import yfinance as yf
from bs4 import BeautifulSoup

def fetch_insider_data():
    """
    Fetch insider trading data from OpenInsider based on user inputs.
    Returns a pandas DataFrame containing the scraped table.
    Assumes that the table contains columns such as 'ticker', 'trade date', 'insider name', and 'price'.
    """
    # -----------------------------
    # Step 1: Get user inputs
    # -----------------------------
    trade_start_input = input("Enter trade start date (YYYY-MM-DD): ").strip()
    trade_end_input = input("Enter trade end date (YYYY-MM-DD): ").strip()
    insider_name = input("Enter insider name (press Enter to skip): ").strip()
    trade_type = input("Enter trade type (purchase, sale, or both): ").strip().lower()

    # -----------------------------
    # Step 2: Convert dates to MM/DD/YYYY and construct tdr parameter
    # -----------------------------
    try:
        trade_start_dt = datetime.strptime(trade_start_input, '%Y-%m-%d')
        trade_end_dt = datetime.strptime(trade_end_input, '%Y-%m-%d')
    except ValueError as e:
        print("Error converting dates:", e)
        exit()

    trade_start_str = trade_start_dt.strftime("%m/%d/%Y")
    trade_end_str = trade_end_dt.strftime("%m/%d/%Y")
    tdr_param = f"{trade_start_str} - {trade_end_str}"

    # -----------------------------
    # Step 3: Build the query URL
    # -----------------------------
    base_url = "http://openinsider.com/screener"
    params = {
        "s": "",
        "o": "",
        "pl": "",
        "ph": "",
        "ll": "",
        "lh": "",
        "fd": "0",
        "fdr": "",
        "td": "-1",
        "tdr": tdr_param,
        "fdlyl": "",
        "fdlyh": "",
        "daysago": "",
        "vl": "",
        "vh": "",
        "ocl": "",
        "och": "",
        "sic1": "-1",
        "sicl": "100",
        "sich": "9999",
        "grp": "0",
        "nfl": "",
        "nfh": "",
        "nil": "",
        "nih": "",
        "nol": "",
        "noh": "",
        "v2l": "",
        "v2h": "",
        "oc2l": "",
        "oc2h": "",
        "sortcol": "0",
        "cnt": "1000",
        "page": "1"
    }

    if insider_name:
        params["pn"] = insider_name  # 'pn' is the insider name parameter

    if trade_type == "purchase":
        params["xp"] = "1"
    elif trade_type == "sale":
        params["xs"] = "1"
    elif trade_type == "both":
        params["xp"] = "1"
        params["xs"] = "1"
    else:
        print("Invalid trade type. Please enter 'purchase', 'sale', or 'both'.")
        exit()

    query_string = urllib.parse.urlencode(params)
    url = f"{base_url}?{query_string}"
    print("Fetching insider trading data from:", url)

    # -----------------------------
    # Step 4: Fetch and parse the page
    # -----------------------------
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve insider trading data. HTTP status code: {response.status_code}")
        exit()

    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table", {"class": "tinytable"})
    if not table:
        print("Could not find the data table on the page.")
        exit()

    rows = table.find_all("tr")
    if len(rows) < 2:
        print("The table does not appear to have data rows.")
        exit()

    # -----------------------------
    # Step 5: Extract headers and data rows
    # -----------------------------
    header_row = rows[0]
    column_names = [cell.text.strip() for cell in header_row.find_all("th")]
    if not column_names:  # Fallback if <th> tags are not used
        column_names = [cell.text.strip() for cell in header_row.find_all("td")]

    data = []
    for row in rows[1:]:
        cols = row.find_all("td")
        if len(cols) == len(column_names):
            data.append([col.text.strip() for col in cols])

    if not data:
        print("No insider trading data rows found.")
        exit()

    df = pd.DataFrame(data, columns=column_names)
    # Normalize column names: replace non-breaking spaces, strip, and lowercase.
    df.columns = [col.strip().replace('\xa0', ' ').lower() for col in df.columns]

    print("\nInsider Trading Data Preview:")
    print(df.head())
    print("\nColumns found:", df.columns.tolist())

    return df

def get_price_on_or_after(df, target):
    """
    Given a DataFrame with a DateTimeIndex and a target date,
    return the closing price and actual date of the first trading day
    on or after the target date.
    """
    target_ts = pd.Timestamp(target)
    available = df[df.index >= target_ts]
    if not available.empty:
        actual_date = available.index[0].date()
        price = available.iloc[0]['Close']
        if isinstance(price, pd.Series):
            price = price.iloc[0]
        return price, actual_date
    return None, None

def get_stock_prices(ticker, start_date):
    """
    Given a stock ticker and a starting date (as a date object),
    download historical data using yfinance and return a dictionary
    of closing prices for several time offsets relative to start_date.
    """
    offsets = {
        "starting date": start_date,
        "3 days": start_date + timedelta(days=3),
        "1 week": start_date + timedelta(weeks=1),
        "2 weeks": start_date + timedelta(weeks=2),
        "1 month": start_date + relativedelta(months=1),
        "3 months": start_date + relativedelta(months=3),
        "6 months": start_date + relativedelta(months=6),
        "1 year": start_date + relativedelta(years=1)
    }
    last_date = max(offsets.values()) + timedelta(days=5)

    data = yf.download(
        ticker,
        start=start_date.strftime("%Y-%m-%d"),
        end=last_date.strftime("%Y-%m-%d")
    )

    if data.empty:
        print(f"No historical data found for {ticker}.")
        return None

    prices = {}
    for label, target_date in offsets.items():
        price, actual_date = get_price_on_or_after(data, target_date)
        prices[label] = {"target": target_date, "actual": actual_date, "price": price}
    return prices

def main():
    # Fetch insider trading data.
    insider_df = fetch_insider_data()

    # We expect the normalized columns to include 'ticker', 'trade date', and 'price'
    if "ticker" not in insider_df.columns or "trade date" not in insider_df.columns or "price" not in insider_df.columns:
        print("Expected columns 'ticker', 'trade date', and 'price' not found in data. Please check the DataFrame columns above.")
        return

    # List to hold each enriched row (original insider row plus price details).
    final_rows = []

    # Offsets we expect (and the order we want them in the CSV).
    offsets_list = ["starting date", "3 days", "1 week", "2 weeks", "1 month", "3 months", "6 months", "1 year"]

    # Iterate over each insider trade row.
    for idx, row in insider_df.iterrows():
        ticker = row.get("ticker")
        trade_date_str = row.get("trade date")
        insider = row.get("insider name") or "N/A"

        # Copy the original row into a dictionary.
        row_data = row.to_dict()

        if not ticker or not trade_date_str:
            print(f"Row {idx} is missing ticker or trade date. Skipping.")
            continue

        # Try parsing the trade date using multiple formats.
        trade_date = None
        for fmt in ("%m/%d/%Y", "%Y-%m-%d"):
            try:
                trade_date = datetime.strptime(trade_date_str, fmt).date()
                break
            except ValueError:
                continue

        if trade_date is None:
            print(f"Error parsing trade date '{trade_date_str}' for ticker {ticker}. Skipping.")
            continue

        prices = get_stock_prices(ticker, trade_date)
        if prices is None:
            # If no price data, add None for all new columns.
            for offset in offsets_list:
                col_price = f"{offset.replace(' ', '_')}_price"
                col_pct = f"{offset.replace(' ', '_')}_percent_gain"
                row_data[col_price] = None
                row_data[col_pct] = None
            final_rows.append(row_data)
            continue

        # Retrieve base price (from yfinance "starting date").
        base_info = prices.get("starting date")
        if base_info is None or base_info.get("price") is None:
            print("No starting price available from yfinance. Skipping net gain calculation.")
            for offset in offsets_list:
                col_price = f"{offset.replace(' ', '_')}_price"
                col_pct = f"{offset.replace(' ', '_')}_percent_gain"
                row_data[col_price] = None
                row_data[col_pct] = None
            final_rows.append(row_data)
            continue

        base_price = base_info["price"]

        # --- New Filtering Step ---
        # Compare the insider trade's reported price to the yfinance starting date price.
        # We assume the insider trade's price is in the "price" field.
        insider_price_str = row.get("price", "")
        try:
            # Remove any $ or commas before conversion.
            insider_price = float(insider_price_str.replace("$", "").replace(",", ""))
        except Exception as e:
            print(f"Skipping trade for {ticker} on {trade_date} because insider trade price '{insider_price_str}' is invalid.")
            continue

        percent_diff = abs((base_price - insider_price) / insider_price) * 100
        if percent_diff > 15:
            print(f"Skipping trade for {ticker} on {trade_date} because insider trade price ${insider_price:.2f} and yfinance price ${base_price:.2f} differ by {percent_diff:.2f}% (>15%).")
            continue

        # For each offset, add the price and computed percent gain columns.
        for offset in offsets_list:
            info = prices.get(offset, {})
            price = info.get("price")
            col_price = f"{offset.replace(' ', '_')}_price"
            col_pct = f"{offset.replace(' ', '_')}_percent_gain"
            row_data[col_price] = price
            if price is not None:
                if offset == "starting date":
                    row_data[col_pct] = 0.0
                else:
                    net_gain = price - base_price
                    percent_gain = (net_gain / base_price) * 100
                    row_data[col_pct] = percent_gain
            else:
                row_data[col_pct] = None

        final_rows.append(row_data)

    # Create a new DataFrame with the enriched data.
    final_df = pd.DataFrame(final_rows)

    # --- Formatting the price and percent gain columns ---
    for offset in offsets_list:
        price_col = f"{offset.replace(' ', '_')}_price"
        pct_col = f"{offset.replace(' ', '_')}_percent_gain"
        final_df[price_col] = final_df[price_col].apply(
            lambda x: f"${float(x):.2f}" if pd.notnull(x) else ""
        )
        final_df[pct_col] = final_df[pct_col].apply(
            lambda x: f"{float(x):.2f}%" if pd.notnull(x) else ""
        )

    # Optionally, preview the final DataFrame.
    print("Final Data Preview:")
    print(final_df.head())

    # Write the final DataFrame to a CSV file.
    output_filename = "insider_trading_with_prices.csv"
    final_df.to_csv(output_filename, index=False)
    print(f"Data written to {output_filename}")

if __name__ == "__main__":
    main()


Enter trade start date (YYYY-MM-DD): 2023-01-01
Enter trade end date (YYYY-MM-DD): 2024-01-01
Enter insider name (press Enter to skip): 
Enter trade type (purchase, sale, or both): purchase
Fetching insider trading data from: http://openinsider.com/screener?s=&o=&pl=&ph=&ll=&lh=&fd=0&fdr=&td=-1&tdr=01%2F01%2F2023+-+01%2F01%2F2024&fdlyl=&fdlyh=&daysago=&vl=&vh=&ocl=&och=&sic1=-1&sicl=100&sich=9999&grp=0&nfl=&nfh=&nil=&nih=&nol=&noh=&v2l=&v2h=&oc2l=&oc2h=&sortcol=0&cnt=1000&page=1&xp=1


[*********************100%***********************]  1 of 1 completed


Insider Trading Data Preview:
    x          filing date  trade date ticker  \
0  AM  2025-01-23 13:59:27  2023-12-12   RNGE   
1   M  2025-01-16 09:01:15  2023-05-22   BLTH   
2   A  2025-01-15 16:15:35  2023-09-13   BHRB   
3   M  2025-01-06 07:35:16  2024-01-02   USEG   
4   M  2024-12-19 14:38:09  2023-06-09    ONB   

                               company name       insider name title  \
0                        Range Impact, Inc.  Loconti Joseph E.   10%   
1          American Battery Materials, Inc.  Suckling Andrew P   Dir   
2  Burke & Herbert Financial Services Corp.  Riojas Jose David   Dir   
3                            Us Energy Corp   Smith Ryan Lewis   CEO   
4                 Old National Bancorp /in/    Rudnick Ellen A   Dir   

     trade type   price       qty      owned δown     value 1d 1w 1m 6m  
0  P - Purchase   $0.17  +130,726  4,842,377  +3%  +$22,102              
1  P - Purchase   $1.35   +11,112     11,112  New  +$15,001              
2  P - Purchase  $4


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Skipping trade for BLTH on 2023-05-22 because insider trade price $1.35 and yfinance price $11.70 differ by 766.67% (>15%).


...

No historical data found for KA.


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Skipping trade for SVVC on 2023-12-04 because insider trade price $0.33 and yfinance price $0.28 differ by 15.15% (>15%).


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Skipping trade for EYPT on 2023-12-04 because insider trade price $21.82 and yfinance price $18.34 differ by 15.95% (>15%).
Final Data Preview:
    x          filing date  trade date ticker  \
0  AM  2025-01-23 13:59:27  2023-12-12   RNGE   
1   A  2025-01-15 16:15:35  2023-09-13   BHRB   
2   M  2024-12-19 14:38:09  2023-06-09    ONB   
3      2024-12-13 16:34:09  2023-06-05    MHI   
4      2024-12-06 09:12:23  2023-10-09   TGEN   

                               company name               insider name title  \
0                        Range Impact, Inc.          Loconti Joseph E.   10%   
1  Burke & Herbert Financial Services Corp.          Riojas Jose David   Dir   
2                 Old National Bancorp /in/            Rudnick Ellen A   Dir   
3  Pioneer Municipal High Income Fund, Inc.  Bank of America Corp /de/   10%   
4                              Tecogen Inc.               Lewis Earl R   Dir   

     trade type   price       qty  ... 2_weeks_price 2_weeks_percent_gain  \
0  

In [ ]:
# Script to obtain insider trading data

import requests
import pandas as pd
from bs4 import BeautifulSoup
import urllib.parse
from datetime import datetime

# -----------------------------
# Step 1: Get user inputs
# -----------------------------
trade_start_input = input("Enter trade start date (YYYY-MM-DD): ").strip()
trade_end_input = input("Enter trade end date (YYYY-MM-DD): ").strip()
insider_name = input("Enter insider name (press Enter to skip): ").strip()
trade_type = input("Enter trade type (purchase, sale, or both): ").strip().lower()

# -----------------------------
# Step 2: Convert dates to MM/DD/YYYY and construct tdr parameter
# -----------------------------
try:
    trade_start_dt = datetime.strptime(trade_start_input, '%Y-%m-%d')
    trade_end_dt = datetime.strptime(trade_end_input, '%Y-%m-%d')
except ValueError as e:  # Catch specific ValueError for date parsing
    print("Error converting dates:", e)
    exit()

trade_start_str = trade_start_dt.strftime("%m/%d/%Y")
trade_end_str = trade_end_dt.strftime("%m/%d/%Y")
# The tdr parameter takes the format "MM/DD/YYYY - MM/DD/YYYY"
tdr_param = f"{trade_start_str} - {trade_end_str}"

# -----------------------------
# Step 3: Build the query URL
# -----------------------------
base_url = "http://openinsider.com/screener"
params = {
    "s": "",
    "o": "",
    "pl": "",
    "ph": "",
    "ll": "",
    "lh": "",
    "fd": "0",       # Filing Date parameter left as default
    "fdr": "",
    "td": "-1",      # Indicates that trade date filtering is active
    "tdr": tdr_param,
    "fdlyl": "",
    "fdlyh": "",
    "daysago": "",
    "vl": "",
    "vh": "",
    "ocl": "",
    "och": "",
    "sic1": "-1",
    "sicl": "100",
    "sich": "9999",
    "grp": "0",
    "nfl": "",
    "nfh": "",
    "nil": "",
    "nih": "",
    "nol": "",
    "noh": "",
    "v2l": "",
    "v2h": "",
    "oc2l": "",
    "oc2h": "",
    "sortcol": "0",
    "cnt": "100",
    "page": "1"
}

if insider_name:
    params["pn"] = insider_name  # 'pn' is the insider name parameter

if trade_type == "purchase":
    params["xp"] = "1"
elif trade_type == "sale":
    params["xs"] = "1"
elif trade_type == "both":
    params["xp"] = "1"
    params["xs"] = "1"
else:
    print("Invalid trade type. Please enter 'purchase', 'sale', or 'both'.")
    exit()


# URL-encode the parameters and create the full URL.
query_string = urllib.parse.urlencode(params)
url = f"{base_url}?{query_string}"
print("Fetching data from:", url)

# -----------------------------
# Step 4: Fetch and parse the page
# -----------------------------
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
if response.status_code != 200:
    print(f"Failed to retrieve data. HTTP status code: {response.status_code}")
    exit()

soup = BeautifulSoup(response.text, "html.parser")
table = soup.find("table", {"class": "tinytable"})
if not table:
    print("Could not find the data table on the page.")
    exit()

rows = table.find_all("tr")
if len(rows) < 2:
    print("The table does not appear to have data rows.")
    exit()

# -----------------------------
# Step 5: Extract headers and data rows
# -----------------------------
# Extract column labels from the header row.
header_row = rows[0]
column_names = [cell.text.strip() for cell in header_row.find_all("th")]
if not column_names:  # Fallback if <th> tags are not used
    column_names = [cell.text.strip() for cell in header_row.find_all("td")]

# Now extract the data rows (skip the header row)
data = []
for row in rows[1:]:
    cols = row.find_all("td")
    # Only add rows with the expected number of columns
    if len(cols) == len(column_names):
        data.append([col.text.strip() for col in cols])

if not data:
    print("No data rows found.")
    exit()

# Create the DataFrame and strip any extra whitespace from column names.
df = pd.DataFrame(data, columns=column_names)
df.columns = [col.strip() for col in df.columns]

# -----------------------------
# Step 6: Display and save the DataFrame
# -----------------------------
print("\nDataFrame preview:")
print(df.head())

csv_filename = "insider_trading_data.csv"
df.to_csv(csv_filename, index=False)
print(f"\nCSV file saved as '{csv_filename}'")

Enter trade start date (YYYY-MM-DD): 2023-01-01
Enter trade end date (YYYY-MM-DD): 2024-01-01
Enter insider name (press Enter to skip): 
Enter trade type (purchase, sale, both): purchase


<ipython-input-2-bcec4e894801>:43: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Error processing AEI: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error processing BSX: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error processing NSP: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Error processing TCBX: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error processing ARCC: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().


...

Error processing GABC: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error processing GABC: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Error processing STRW: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error processing NYC: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error processing GDRZF: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().


[*********************100%***********************]  1 of 1 completed


Error processing AEI: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
CSV file created successfully!


In [ ]:
# Script to get the price of a stock at some time points after a given date

import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import yfinance as yf

def get_price_on_or_after(df, target):
    """
    Given a DataFrame with a DateTimeIndex and a target date,
    return the closing price and actual date of the first trading day
    on or after the target date.
    """
    target_ts = pd.Timestamp(target)
    available = df[df.index >= target_ts]
    if not available.empty:
        actual_date = available.index[0].date()
        price = available.iloc[0]['Close']
        # If the price is a Series (due to MultiIndex columns), extract the scalar.
        if isinstance(price, pd.Series):
            price = price.iloc[0]
        return price, actual_date
    return None, None

def main():
    # Request user input.
    ticker_sym = input("Enter stock ticker (e.g., AAPL): ")
    start_str = input("Enter starting date (YYYY-MM-DD): ")

    # Convert the input date string into a date object.
    try:
        start_date = datetime.datetime.strptime(start_str, "%Y-%m-%d").date()
    except ValueError:
        print("Invalid date format. Please use YYYY-MM-DD.")
        return

    # Compute target dates relative to the starting date.
    offsets = {
        "Starting Date": start_date,
        "3 Days": start_date + datetime.timedelta(days=3),
        "1 Week": start_date + datetime.timedelta(weeks=1),
        "2 Weeks": start_date + datetime.timedelta(weeks=2),
        "1 Month": start_date + relativedelta(months=1),
        "3 Months": start_date + relativedelta(months=3),
        "6 Months": start_date + relativedelta(months=6),
        "1 Year": start_date + relativedelta(years=1)
    }

    # Download historical data from the starting date until a few days after the furthest target date.
    last_date = max(offsets.values()) + datetime.timedelta(days=5)
    data = yf.download(
        ticker_sym,
        start=start_date.strftime("%Y-%m-%d"),
        end=last_date.strftime("%Y-%m-%d")
    )

    if data.empty:
        print("No historical data found. Please check the ticker and date range.")
        return

    print(f"\nPrices for {ticker_sym}:")
    for label, target_date in offsets.items():
        price, actual_date = get_price_on_or_after(data, target_date)
        if price is not None:
            print(f"{label} (target: {target_date}, actual: {actual_date}): ${price:.2f}")
        else:
            print(f"{label} (target: {target_date}): No data available.")

# Run the main function.
main()


Enter stock ticker (e.g., AAPL): aapl
Enter starting date (YYYY-MM-DD): 2022-01-01


[*********************100%***********************]  1 of 1 completed


Prices for aapl:
Starting Date (target: 2022-01-01, actual: 2022-01-03): $178.88
3 Days (target: 2022-01-04, actual: 2022-01-04): $176.61
1 Week (target: 2022-01-08, actual: 2022-01-10): $169.23
2 Weeks (target: 2022-01-15, actual: 2022-01-18): $166.88
1 Month (target: 2022-02-01, actual: 2022-02-01): $171.61
3 Months (target: 2022-04-01, actual: 2022-04-01): $171.53
6 Months (target: 2022-07-01, actual: 2022-07-01): $136.92
1 Year (target: 2023-01-01, actual: 2023-01-03): $123.63


TODO

Note: "time step" = an arbitrary time (e.g. 1 week) after trade
* EDA
    - Call .describe() on each column of data
* Structure of data
    - Rows: Insider trades
        - Can group by insider name and examine mean of percent gain at each time step
    - Cols: Insider trade info + stock price/percent gains at each time step




In [ ]:
!pip install yfinance python-dateutil bs4 requests


In [ ]:
# Test script to examine stock price at fixed time points after insider trade

import requests
import pandas as pd
import urllib.parse
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import yfinance as yf
from bs4 import BeautifulSoup

def fetch_insider_data():
    """
    Fetch insider trading data from OpenInsider based on user inputs.
    Returns a pandas DataFrame containing the scraped table.
    Assumes that the table contains columns such as 'ticker', 'trade date', and 'insider name'.
    """
    # -----------------------------
    # Step 1: Get user inputs
    # -----------------------------
    trade_start_input = input("Enter trade start date (YYYY-MM-DD): ").strip()
    trade_end_input = input("Enter trade end date (YYYY-MM-DD): ").strip()
    insider_name = input("Enter insider name (press Enter to skip): ").strip()
    trade_type = input("Enter trade type (purchase, sale, or both): ").strip().lower()

    # -----------------------------
    # Step 2: Convert dates to MM/DD/YYYY and construct tdr parameter
    # -----------------------------
    try:
        trade_start_dt = datetime.strptime(trade_start_input, '%Y-%m-%d')
        trade_end_dt = datetime.strptime(trade_end_input, '%Y-%m-%d')
    except ValueError as e:
        print("Error converting dates:", e)
        exit()

    trade_start_str = trade_start_dt.strftime("%m/%d/%Y")
    trade_end_str = trade_end_dt.strftime("%m/%d/%Y")
    tdr_param = f"{trade_start_str} - {trade_end_str}"

    # -----------------------------
    # Step 3: Build the query URL
    # -----------------------------
    base_url = "http://openinsider.com/screener"
    params = {
        "s": "",
        "o": "",
        "pl": "",
        "ph": "",
        "ll": "",
        "lh": "",
        "fd": "0",
        "fdr": "",
        "td": "-1",
        "tdr": tdr_param,
        "fdlyl": "",
        "fdlyh": "",
        "daysago": "",
        "vl": "",
        "vh": "",
        "ocl": "",
        "och": "",
        "sic1": "-1",
        "sicl": "100",
        "sich": "9999",
        "grp": "0",
        "nfl": "",
        "nfh": "",
        "nil": "",
        "nih": "",
        "nol": "",
        "noh": "",
        "v2l": "",
        "v2h": "",
        "oc2l": "",
        "oc2h": "",
        "sortcol": "0",
        "cnt": "100",
        "page": "1"
    }

    if insider_name:
        params["pn"] = insider_name  # 'pn' is the insider name parameter

    if trade_type == "purchase":
        params["xp"] = "1"
    elif trade_type == "sale":
        params["xs"] = "1"
    elif trade_type == "both":
        params["xp"] = "1"
        params["xs"] = "1"
    else:
        print("Invalid trade type. Please enter 'purchase', 'sale', or 'both'.")
        exit()

    query_string = urllib.parse.urlencode(params)
    url = f"{base_url}?{query_string}"
    print("Fetching insider trading data from:", url)

    # -----------------------------
    # Step 4: Fetch and parse the page
    # -----------------------------
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve insider trading data. HTTP status code: {response.status_code}")
        exit()

    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table", {"class": "tinytable"})
    if not table:
        print("Could not find the data table on the page.")
        exit()

    rows = table.find_all("tr")
    if len(rows) < 2:
        print("The table does not appear to have data rows.")
        exit()

    # -----------------------------
    # Step 5: Extract headers and data rows
    # -----------------------------
    header_row = rows[0]
    column_names = [cell.text.strip() for cell in header_row.find_all("th")]
    if not column_names:  # Fallback if <th> tags are not used
        column_names = [cell.text.strip() for cell in header_row.find_all("td")]

    data = []
    for row in rows[1:]:
        cols = row.find_all("td")
        if len(cols) == len(column_names):
            data.append([col.text.strip() for col in cols])

    if not data:
        print("No insider trading data rows found.")
        exit()

    df = pd.DataFrame(data, columns=column_names)
    # Normalize column names: replace non-breaking spaces, strip, and lowercase.
    df.columns = [col.strip().replace('\xa0', ' ').lower() for col in df.columns]

    print("\nInsider Trading Data Preview:")
    print(df.head())
    print("\nColumns found:", df.columns.tolist())

    return df

def get_price_on_or_after(df, target):
    """
    Given a DataFrame with a DateTimeIndex and a target date,
    return the closing price and actual date of the first trading day
    on or after the target date.
    """
    target_ts = pd.Timestamp(target)
    available = df[df.index >= target_ts]
    if not available.empty:
        actual_date = available.index[0].date()
        price = available.iloc[0]['Close']
        if isinstance(price, pd.Series):
            price = price.iloc[0]
        return price, actual_date
    return None, None

def get_stock_prices(ticker, start_date):
    """
    Given a stock ticker and a starting date (as a date object),
    download historical data using yfinance and return a dictionary
    of closing prices for several time offsets relative to start_date.
    """
    offsets = {
        "starting date": start_date,
        "3 days": start_date + timedelta(days=3),
        "1 week": start_date + timedelta(weeks=1),
        "2 weeks": start_date + timedelta(weeks=2),
        "1 month": start_date + relativedelta(months=1),
        "3 months": start_date + relativedelta(months=3),
        "6 months": start_date + relativedelta(months=6),
        "1 year": start_date + relativedelta(years=1)
    }
    last_date = max(offsets.values()) + timedelta(days=5)

    data = yf.download(
        ticker,
        start=start_date.strftime("%Y-%m-%d"),
        end=last_date.strftime("%Y-%m-%d")
    )

    if data.empty:
        print(f"No historical data found for {ticker}.")
        return None

    prices = {}
    for label, target_date in offsets.items():
        price, actual_date = get_price_on_or_after(data, target_date)
        prices[label] = {"target": target_date, "actual": actual_date, "price": price}
    return prices

def main():
    # Fetch insider trading data.
    insider_df = fetch_insider_data()
    print("\nInsider Trading Data Preview:\n")
    print(insider_df.info())


    # We expect the normalized columns to include 'ticker' and 'trade date'
    if "ticker" not in insider_df.columns or "trade date" not in insider_df.columns:
        print("Expected columns 'ticker' and 'trade date' not found in data. Please check the DataFrame columns above.")
        return

    # Iterate over each row in the insider data.
    for idx, row in insider_df.iterrows():
        ticker = row.get("ticker")
        trade_date_str = row.get("trade date")
        insider = row.get("insider name") or "N/A"

        if not ticker or not trade_date_str:
            print(f"Row {idx} is missing ticker or trade date. Skipping.")
            continue

        # Try parsing the trade date using multiple formats.
        trade_date = None
        for fmt in ("%m/%d/%Y", "%Y-%m-%d"):
            try:
                trade_date = datetime.strptime(trade_date_str, fmt).date()
                break
            except ValueError:
                continue

        if trade_date is None:
            print(f"Error parsing trade date '{trade_date_str}' for ticker {ticker}. Skipping.")
            continue

        # Print a visual barrier and header for this query.
        print("\n" + "=" * 60)
        print(f"Ticker: {ticker} | Trade Date: {trade_date} | Insider: {insider}")

        prices = get_stock_prices(ticker, trade_date)
        if prices is None:
            continue

        base_info = prices.get("starting date")
        if base_info is None or base_info.get("price") is None:
            print("No starting price available. Skipping net gain calculation.")
            continue

        base_price = base_info["price"]
        print(f"Starting Date Price: ${base_price:.2f}\n")

        net_gain_occurred = False

        # Loop over all offsets (skipping the base 'starting date' in the net gain details).
        for label, info in prices.items():
            if label.lower() == "starting date":
                # Already printed the base price.
                continue

            price = info.get("price")
            target_date = info.get("target")
            actual_date = info.get("actual")
            if price is None:
                print(f"{label.title()} (target: {target_date}, actual: {actual_date}): No data available.")
            else:
                net_gain = price - base_price
                percent_gain = (net_gain / base_price) * 100
                # Use a green check if net gain is positive, else a red X.
                symbol = "✅" if net_gain > 0 else "❌"
                if net_gain > 0:
                    net_gain_occurred = True
                print(f"{label.title()} (target: {target_date}, actual: {actual_date}): Price: ${price:.2f} | Net Gain: ${net_gain:.2f} | Percent Gain %{percent_gain:.2f} {symbol}")

        overall_symbol = "✅" if net_gain_occurred else "❌"
        print(f"\nOverall Net Gain Occurred? {overall_symbol}")
        print("=" * 60 + "\n")

if __name__ == "__main__":
    main()


Enter trade start date (YYYY-MM-DD): 2020-05-01
Enter trade end date (YYYY-MM-DD): 2021-05-01
Enter insider name (press Enter to skip): 
Enter trade type (purchase, sale, or both): both
Fetching insider trading data from: http://openinsider.com/screener?s=&o=&pl=&ph=&ll=&lh=&fd=0&fdr=&td=-1&tdr=05%2F01%2F2020+-+05%2F01%2F2021&fdlyl=&fdlyh=&daysago=&vl=&vh=&ocl=&och=&sic1=-1&sicl=100&sich=9999&grp=0&nfl=&nfh=&nil=&nih=&nol=&noh=&v2l=&v2h=&oc2l=&oc2h=&sortcol=0&cnt=100&page=1&xp=1&xs=1


[*********************100%***********************]  1 of 1 completed


Insider Trading Data Preview:
   x          filing date  trade date ticker  \
0     2025-01-17 21:49:11  2020-06-24   DAKT   
1  M  2024-12-13 16:16:33  2020-06-10    VCV   
2  M  2024-12-13 16:16:33  2020-06-10    VCV   
3  M  2024-12-13 09:20:29  2021-04-16   CMTV   
4  M  2024-10-25 16:30:03  2020-12-11   VTRS   

                                      company name               insider name  \
0                              Daktronics Inc /sd/             Gatzke Carla S   
1  Invesco California Value Municipal Income Trust  Bank of America Corp /de/   
2  Invesco California Value Municipal Income Trust  Bank of America Corp /de/   
3                            Community Bancorp /vt          Moore Jeffrey Lee   
4                                      Viatris Inc               Korman Harry   

                         title    trade type   price     qty    owned   δown  \
0  Secretary, VP of Human Reso  P - Purchase   $4.32    +435  842,940     0%   
1                          10%   


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Starting Date Price: $9.27

3 Days (target: 2020-06-13, actual: 2020-06-15): Price: $9.34 | Net Gain: $0.07 | Percent Gain %0.71 ✅
1 Week (target: 2020-06-17, actual: 2020-06-17): Price: $9.30 | Net Gain: $0.03 | Percent Gain %0.28 ✅
2 Weeks (target: 2020-06-24, actual: 2020-06-24): Price: $9.32 | Net Gain: $0.04 | Percent Gain %0.46 ✅
1 Month (target: 2020-07-10, actual: 2020-07-10): Price: $9.55 | Net Gain: $0.27 | Percent Gain %2.95 ✅
3 Months (target: 2020-09-10, actual: 2020-09-10): Price: $10.01 | Net Gain: $0.74 | Percent Gain %7.97 ✅
6 Months (target: 2020-12-10, actual: 2020-12-10): Price: $10.55 | Net Gain: $1.28 | Percent Gain %13.79 ✅
1 Year (target: 2021-06-10, actual: 2021-06-10): Price: $11.51 | Net Gain: $2.24 | Percent Gain %24.14 ✅

Overall Net Gain Occurred? ✅


Ticker: VCV | Trade Date: 2020-06-10 | Insider: Bank of America Corp /de/
Starting Date Price: $9.27

3 Days (target: 2020-06-13, actual: 2020-06-15): Price: $9.34 | Net Gain: $0.07 | Percent Gain %0.71 ✅
1 W

...

Starting Date Price: $461.40

3 Days (target: 2020-09-20, actual: 2020-09-21): Price: $456.00 | Net Gain: $-5.40 | Percent Gain %-1.17 ❌
1 Week (target: 2020-09-24, actual: 2020-09-24): Price: $554.40 | Net Gain: $93.00 | Percent Gain %20.16 ✅
2 Weeks (target: 2020-10-01, actual: 2020-10-01): Price: $584.80 | Net Gain: $123.40 | Percent Gain %26.74 ✅
1 Month (target: 2020-10-17, actual: 2020-10-19): Price: $697.60 | Net Gain: $236.20 | Percent Gain %51.19 ✅
3 Months (target: 2020-12-17, actual: 2020-12-17): Price: $566.80 | Net Gain: $105.40 | Percent Gain %22.84 ✅
6 Months (target: 2021-03-17, actual: 2021-03-17): Price: $420.80 | Net Gain: $-40.60 | Percent Gain %-8.80 ❌
1 Year (target: 2021-09-17, actual: 2021-09-17): Price: $225.00 | Net Gain: $-236.40 | Percent Gain %-51.24 ❌

Overall Net Gain Occurred? ✅


Ticker: AMWL | Trade Date: 2020-09-17 | Insider: Schlegel Stephen J.
Starting Date Price: $461.40

3 Days (target: 2020-09-20, actual: 2020-09-21): Price: $456.00 | Net Gain: $

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Starting Date Price: $14.11

3 Days (target: 2021-03-26, actual: 2021-03-26): Price: $13.30 | Net Gain: $-0.81 | Percent Gain %-5.74 ❌
1 Week (target: 2021-03-30, actual: 2021-03-30): Price: $12.30 | Net Gain: $-1.81 | Percent Gain %-12.83 ❌
2 Weeks (target: 2021-04-06, actual: 2021-04-06): Price: $14.24 | Net Gain: $0.13 | Percent Gain %0.92 ✅
1 Month (target: 2021-04-23, actual: 2021-04-23): Price: $13.18 | Net Gain: $-0.93 | Percent Gain %-6.59 ❌
3 Months (target: 2021-06-23, actual: 2021-06-23): Price: $11.13 | Net Gain: $-2.98 | Percent Gain %-21.12 ❌
6 Months (target: 2021-09-23, actual: 2021-09-23): Price: $11.31 | Net Gain: $-2.80 | Percent Gain %-19.84 ❌
1 Year (target: 2022-03-23, actual: 2022-03-23): Price: $2.46 | Net Gain: $-11.65 | Percent Gain %-82.57 ❌

Overall Net Gain Occurred? ✅


Ticker: SNSE | Trade Date: 2021-02-18 | Insider: Peyer James
Starting Date Price: $19.92

3 Days (target: 2021-02-21, actual: 2021-02-22): Price: $18.75 | Net Gain: $-1.17 | Percent Gain %-

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Starting Date Price: $13.66

3 Days (target: 2020-12-20, actual: 2020-12-21): Price: $15.09 | Net Gain: $1.43 | Percent Gain %10.43 ✅
1 Week (target: 2020-12-24, actual: 2020-12-24): Price: $14.15 | Net Gain: $0.49 | Percent Gain %3.62 ✅
2 Weeks (target: 2020-12-31, actual: 2020-12-31): Price: $15.48 | Net Gain: $1.82 | Percent Gain %13.34 ✅
1 Month (target: 2021-01-17, actual: 2021-01-19): Price: $18.38 | Net Gain: $4.72 | Percent Gain %34.56 ✅
3 Months (target: 2021-03-17, actual: 2021-03-17): Price: $6.56 | Net Gain: $-7.10 | Percent Gain %-51.95 ❌
6 Months (target: 2021-06-17, actual: 2021-06-17): Price: $9.69 | Net Gain: $-3.97 | Percent Gain %-29.03 ❌
1 Year (target: 2021-12-17, actual: 2021-12-17): Price: $14.31 | Net Gain: $0.65 | Percent Gain %4.78 ✅

Overall Net Gain Occurred? ✅


Ticker: JSDA | Trade Date: 2020-05-20 | Insider: Sirkin Clive M
Starting Date Price: $0.18

3 Days (target: 2020-05-23, actual: 2020-05-26): Price: $0.17 | Net Gain: $-0.01 | Percent Gain %-7.10 ❌
1